# GPT-3.5-Turbo on GSM8K

In [68]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [69]:
openai.api_key = "sk-"

In [70]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('../gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [16]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [17]:
gsm8k_test = gsm8k['test']

In [18]:
prompt_complex = open('../gsm8k/lib_prompt/prompt_hardest.txt').read()

In [19]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [20]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.chat.completions.create(**kwargs)

In [21]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [22]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [23]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [24]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [25]:
response.choices[0].message.content

'It takes 2 bolts of blue fiber and half that much white fiber, so it takes 2 + (1/2)*2 = 3 bolts in total. \n\nThe answer is 3 bolts.'

In [26]:
print(response.choices[0].message.content)

It takes 2 bolts of blue fiber and half that much white fiber, so it takes 2 + (1/2)*2 = 3 bolts in total. 

The answer is 3 bolts.


# compress 안함

In [37]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 20): break

  1%|▏        | 19/1319 [00:59<1:08:24,  3.16s/it]


In [38]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 20 correct 10 ratio 0.5000


# compress 함

In [71]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor("Qwen/Qwen2-0.5B", device_map="cpu")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [73]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n' 
        prompt_c1 = llm_lingua.compress_prompt(prompt_complex, instruction="", question="", target_token=300)
        prompt_c2 = prompt_c1['compressed_prompt']
        # print("QQQ",prompt_q)
        print("CCC\n",prompt_c1)
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_c2},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 20): break

  0%|                                    | 0/1319 [00:00<?, ?it/s]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  0%|                         | 1/1319 [00:33<12:24:38, 33.90s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  0%|                         | 2/1319 [01:03<11:23:12, 31.13s/it]

...

  1%|▎                       | 18/1319 [10:41<13:23:32, 37.06s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  1%|▎                       | 19/1319 [11:18<13:19:23, 36.90s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  1%|▎                       | 19/1319 [11:53<13:33:12, 37.53s/it]


In [75]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 20 correct 0 ratio 0.0000
